<a href="https://colab.research.google.com/github/nbisso/TP2-Orga-7506/blob/main/TP_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#https://realpython.com/train-test-split-python-data/

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df_train_labels = pd.read_csv('/content/drive/MyDrive/datasets_tp2/train_labels.csv', index_col = 'building_id')

In [12]:
df_train_values = pd.read_csv('/content/drive/MyDrive/datasets_tp2/train_values.csv', 
                              dtype = {'land_surface_condition':'category', 'foundation_type':'category'}, index_col = 'building_id')

In [13]:
df_train_values.dtypes

geo_level_1_id                               int64
geo_level_2_id                               int64
geo_level_3_id                               int64
count_floors_pre_eq                          int64
age                                          int64
area_percentage                              int64
height_percentage                            int64
land_surface_condition                    category
foundation_type                           category
roof_type                                   object
ground_floor_type                           object
other_floor_type                            object
position                                    object
plan_configuration                          object
has_superstructure_adobe_mud                 int64
has_superstructure_mud_mortar_stone          int64
has_superstructure_stone_flag                int64
has_superstructure_cement_mortar_stone       int64
has_superstructure_mud_mortar_brick          int64
has_superstructure_cement_morta

In [14]:
df_train_labels.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


In [15]:
df_train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [16]:
#df_train_values = df_train_values.set_index('building_id')

In [17]:
columnas = ['foundation_type', 
                     'area_percentage', 
                     'height_percentage',
                     'count_floors_pre_eq',
                     'land_surface_condition',
                     'has_superstructure_cement_mortar_stone', 'damage_grade']

In [18]:
df_train_values['damage_grade'] = df_train_labels['damage_grade']


In [19]:
df = df_train_values[columnas]
df['damage_grade'] = df['damage_grade'] -1
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone,damage_grade
building_id,,,,,,,
802906,r,6,5,2,t,0,2
28830,r,8,7,2,o,0,1
94947,r,5,5,2,t,0,2
590882,r,6,5,2,t,0,1
201944,r,8,9,3,t,0,2


In [20]:
X,y = df.iloc[:, :-1], df.iloc[:, -1]
X

,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone
building_id,,,,,,
802906,r,6,5,2,t,0
28830,r,8,7,2,o,0
94947,r,5,5,2,t,0
590882,r,6,5,2,t,0
201944,r,8,9,3,t,0
...,...,...,...,...,...,...
688636,r,6,3,1,n,0
669485,r,6,5,2,t,0
602512,r,6,7,3,t,0


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.333337170617)


In [22]:
y_test

building_id
83086      2
79035      1
552293     2
640564     1
1051848    2
          ..
604647     1
193680     2
519954     2
860271     1
131845     1
Name: damage_grade, Length: 86868, dtype: int64

In [23]:
lgb_train = lgb.Dataset(X_train, label = y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference= lgb_train)

In [24]:
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_classes': 3,
    'num_leaves': 40,
    'learning_rate': 0.05,
    'max_depth': 8
}

In [25]:
#https://neptune.ai/blog/lightgbm-parameters-guide
model = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's multi_logloss: 0.907625
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 0.901484
[3]	valid_0's multi_logloss: 0.89599
[4]	valid_0's multi_logloss: 0.89103
[5]	valid_0's multi_logloss: 0.886519
[6]	valid_0's multi_logloss: 0.882396
[7]	valid_0's multi_logloss: 0.878608
[8]	valid_0's multi_logloss: 0.875126
[9]	valid_0's multi_logloss: 0.871907
[10]	valid_0's multi_logloss: 0.868925
[11]	valid_0's multi_logloss: 0.866169
[12]	valid_0's multi_logloss: 0.863596
[13]	valid_0's multi_logloss: 0.861208
[14]	valid_0's multi_logloss: 0.858982
[15]	valid_0's multi_logloss: 0.85691
[16]	valid_0's multi_logloss: 0.854963
[17]	valid_0's multi_logloss: 0.853148
[18]	valid_0's multi_logloss: 0.851449
[19]	valid_0's multi_logloss: 0.849856
[20]	valid_0's multi_logloss: 0.848359
[21]	valid_0's multi_logloss: 0.846953
[22]	valid_0's multi_logloss: 0.84563
[23]	valid_0's multi_logloss: 0.844386
[24]	valid_0's multi_logloss: 0.843206
[25]	valid_0's 

In [26]:
pred = model.predict(X_test)

In [27]:
labels_pred = [np.argmax(v) + 1 for v in pred]
len(labels_pred)

86868

In [28]:
error = f1_score(y_test, labels_pred, average ='micro')
error

0.3146613252290832

In [29]:
df_result = pd.DataFrame(data = {'building_id': [], 'damage_grade': []})
df_result['building_id'] = X_test.reset_index()['building_id']
df_result['damage_grade'] = labels_pred
df_result

,building_id,damage_grade
0,83086,2
1,79035,2
2,552293,2
3,640564,2
4,1051848,2
...,...,...
86863,604647,3
86864,193680,2
86865,519954,2
86866,860271,2


In [34]:
df.reset_index()

,building_id,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone,damage_grade
0,802906,r,6,5,2,t,0,2
1,28830,r,8,7,2,o,0,1
2,94947,r,5,5,2,t,0,2
3,590882,r,6,5,2,t,0,1
4,201944,r,8,9,3,t,0,2
...,...,...,...,...,...,...,...,...
260596,688636,r,6,3,1,n,0,1
260597,669485,r,6,5,2,t,0,2
260598,602512,r,6,7,3,t,0,2
260599,151409,r,14,6,2,t,0,1


In [52]:
submission_format = pd.read_csv('/content/drive/MyDrive/datasets_tp2/submission_format.csv', index_col='building_id')



my_submission = pd.DataFrame(data=labels_pred,
                             columns=submission_format.columns,
                             index=submission_format.index)

my_submission

,damage_grade
building_id,
300051,2
99355,2
890251,2
745817,2
421793,2
...,...
310028,3
663567,2
1049160,2


In [54]:
my_submission.to_csv('/content/drive/MyDrive/datasets_tp2/12072021-training-result.csv',\
index= True, header=True,sep=',')